This generates the landmarks for the different face photos shown in the fampics experiment.

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import os
from glob import glob

import numpy as np
import pandas as pd

import skimage.io as io
io.use_plugin('matplotlib')

import moviepy.editor as mpy

import menpo
import menpo.io as mio
from menpo.io import export_image, export_landmark_file
from menpowidgets import visualize_images

from pathlib import Path

from tqdm import tqdm

In [2]:
import cv2
from menpo.shape import PointCloud
from menpo.shape import PointDirectedGraph
from menpo.shape import bounding_box

from menpodetect.dlib import load_dlib_frontal_face_detector
from menpodetect.pico import load_pico_frontal_face_detector
from menpodetect.ffld2 import load_ffld2_frontal_face_detector

dlib_detector = load_dlib_frontal_face_detector()
pico_detector = load_pico_frontal_face_detector()
ffld2_detector = load_ffld2_frontal_face_detector()

from menpofit.dlib import DlibWrapper as dlib_fitter
predictor_path = "/data1/famface01/data/misc/shape_predictor_68_face_landmarks.dat"
dlib = dlib_fitter(predictor_path)

from menpo.landmark import face_ibug_68_to_face_ibug_68_trimesh, face_ibug_68_to_face_ibug_68, labeller

face_parts = {
    'chin': range(17),              # 0:16
    'left-eyebrow': range(17,22),   # 17:21
    'right-eyebrow': range(22,27),  # 22:26
    'nose': range(27,36),           # 27:35
    'left-eye': range(36,42),       # 36:41
    'right-eye': range(42,48),      # 42:47
    'outer-mouth': range(48,60),    # 48:59
    'inner-mouth': range(60,68)     # 60:67
}
face_parts['eyebrow'] = face_parts['left-eyebrow'] + face_parts['right-eyebrow']
face_parts['eye'] = face_parts['left-eye'] + face_parts['right-eye']

blue = (255,0,0)
red  = (0,0,255)
green= (0,255,0)


def add_feature(im, land, part, isClosed, color, *args, **kwrds):
    pts = land[face_parts[part],]
    pts = pts.reshape((-1,1,2))
    cv2.polylines(im, [pts], isClosed, color, *args, **kwrds)

def add_landmarks(img, land, color=red, thick=1):
  add_feature(img, land, 'chin', False, color, thick)
  add_feature(img, land, 'left-eyebrow', False, color, thick)
  add_feature(img, land, 'right-eyebrow', False, color, thick)
  add_feature(img, land, 'nose', False, color, thick)
  add_feature(img, land, 'left-eye', True, color, thick)
  add_feature(img, land, 'right-eye', True, color, thick)
  add_feature(img, land, 'outer-mouth', True, color, thick)
  add_feature(img, land, 'inner-mouth', True, color, thick)
  return img


def image_to_menpo(image):
    menpo_image = menpo.image.Image(np.rollaxis(image, 2, 0))
    return menpo_image

def image_to_menpo_float32(image):
    image = np.rollaxis(image, 2, 0)
    image = menpo.image.base.normalize_pixels_range(image).astype(np.float32)
    menpo_image = menpo.image.Image(image)
    return menpo_image

def menpo_to_image(menpo_image):
    image = np.rollaxis(menpo_image.pixels, 0, 3)
    #np.stack((menpo_image.pixels[0,:,:], menpo_image.pixels[1,:,:], menpo_image.pixels[2,:,:]), axis=-1)
    return image

def add_landmarks_menpo(orig_img, menpo_img, out_menpo=True, *args, **kwrds):
    limg = np.copy(orig_img)

    lands = menpo_img.landmarks['PTS'].lms
    lands = np.fliplr(lands.h_points().T[:,:2])

    limg = add_landmarks(limg, lands, *args, **kwrds)
    if out_menpo: limg = image_to_menpo(limg)
    
    return limg

def add_bounding_box(img, bbox):
    bbox0 = bounding_box(bbox.h_points().T[:,:2][0,:], bbox.h_points().T[:,:2][2,:])
    img.landmarks['bbox_0'] = bbox0
    return bbox0

def find_bounding_box(img, verbose=False):
    pcs = dlib_detector(img, group_prefix='bbox')
    detector = 'dlib'
    if len(pcs) == 0:
        pcs = pico_detector(img, group_prefix='bbox')
        detector = 'pico'
        if len(pcs) == 0:
            pcs = ffld2_detector(img, group_prefix='bbox')
            detector = 'ffld2'
            if len(pcs) == 0:
                if verbose: print('no bbox found')
                detector = None
                pcs = None
    return (pcs, detector)

def find_landmarks(img):
    if 'bbox_0' not in img.landmarks:
        pcs,detector = find_bounding_box(img)
    fit = dlib.fit_from_bb(img, img.landmarks['bbox_0'].lms)
    img.landmarks['PTS'] = fit.final_shape
    return fit

def crop_face(img):
    pc = img.landmarks['PTS'].lms
    imins, imaxs = pc.bounds()
    img2  = img.crop([imins[0]-((imaxs[0]-imins[0])*0.5), imins[1]-((imaxs[1]-imins[1])*0.1)], 
                     [imaxs[0]+((imaxs[0]-imins[0])*0.05), imaxs[1]+((imaxs[1]-imins[1])*0.1)], 
                     True)
    return img2

def zload_vid(ifname):
    small_size = [640,360]
    large_size = [854,480]

    vid = mpy.VideoFileClip(ifname)
    
    # Fix vid size if needed
    if (vid.size[0]<640) | (vid.size[1]<360):
        vid = vid.on_color(size=small_size, color=(0,0,0))
    elif ((vid.size[0]>640) & (vid.size[0]<854)) | ((vid.size[1]>360) & (vid.size[1]<480)):
        vid = vid.on_color(size=large_size, color=(0,0,0))
    elif (vid.size[0]==640) & (vid.size[1]==480):
        vid = vid.on_color(size=large_size, color=(0,0,0))
    
    return vid

# Load

## Paths, etc

In [3]:
fpaths = sorted(glob("/data1/famface01/data/stimuli/pics/fampics_task/*/*.jpg"))
fpaths[:10]

['/data1/famface01/data/stimuli/pics/fampics_task/Angelina_Jolie/Angelina_Jolie_117.jpg',
 '/data1/famface01/data/stimuli/pics/fampics_task/Angelina_Jolie/Angelina_Jolie_128.jpg',
 '/data1/famface01/data/stimuli/pics/fampics_task/Angelina_Jolie/Angelina_Jolie_14.jpg',
 '/data1/famface01/data/stimuli/pics/fampics_task/Angelina_Jolie/Angelina_Jolie_15.jpg',
 '/data1/famface01/data/stimuli/pics/fampics_task/Angelina_Jolie/Angelina_Jolie_153.jpg',
 '/data1/famface01/data/stimuli/pics/fampics_task/Angelina_Jolie/Angelina_Jolie_168.jpg',
 '/data1/famface01/data/stimuli/pics/fampics_task/Angelina_Jolie/Angelina_Jolie_178.jpg',
 '/data1/famface01/data/stimuli/pics/fampics_task/Angelina_Jolie/Angelina_Jolie_181.jpg',
 '/data1/famface01/data/stimuli/pics/fampics_task/Angelina_Jolie/Angelina_Jolie_187.jpg',
 '/data1/famface01/data/stimuli/pics/fampics_task/Angelina_Jolie/Angelina_Jolie_188.jpg']

In [4]:
# Save the associated identities with each file
lst_ids = [ os.path.basename(os.path.dirname(fpath)) for fpath in fpaths ]
uids = np.unique(lst_ids)

## Data, landmark, etc

Save those landmarks.

In [ ]:
images = []
for fpath in tqdm(fpaths):
    # Read
    img = mio.import_image(fpath)
    
    # Bounding Box
    pcs, detector = find_bounding_box(img)
    
    # Landmarks
    find_landmarks(img)
    labeller(img, 'PTS', face_ibug_68_to_face_ibug_68_trimesh)
    labeller(img, 'face_ibug_68_trimesh', face_ibug_68_to_face_ibug_68)
    
    # Save
    images.append(img)
    ofile = "%s/%s" % (os.path.dirname(fpath), os.path.basename(fpath).replace(".jpg", ".pts"))
    export_landmark_file(img.landmarks['PTS'], ofile, overwrite=True)

In [ ]:
visualize_images(images)

### Sample Photo

In [ ]:
i = 0
fpath = fpaths[i]
fid = lst_ids[i]

img = mio.import_image(fpath)
img.view()

In [ ]:
pcs, detector = find_bounding_box(img)

In [ ]:
find_landmarks(img)
labeller(img, 'PTS', face_ibug_68_to_face_ibug_68_trimesh)
labeller(img, 'face_ibug_68_trimesh', face_ibug_68_to_face_ibug_68)

In [ ]:
img.view_landmarks(group='PTS')

In [ ]:
"%s/%s" % (os.path.dirname(fpath), os.path.basename(fpath).replace(".jpg", ".pts"))

# Save Masked

In [7]:
odir = "/data1/famface01/data/stimuli/pics/fampics_exp_masked"
if not os.path.exists(odir): os.mkdir(odir)

In [12]:
for i,fpath in tqdm(enumerate(fpaths), total=len(fpaths)):
    uid = lst_ids[i]
    mdir = "%s/%s" % (odir, uid)
    if not os.path.exists(mdir): os.mkdir(mdir)
    
    # Read
    img = mio.import_image(fpath)
    mimg = img.as_masked().constrain_mask_to_landmarks(group='PTS')
    
    # Save
    img_file = "%s/%s" % (mdir, os.path.basename(fpath))
    pts_file = "%s/%s" % (mdir, os.path.basename(fpath).replace(".jpg", ".pts"))
    
    export_image(mimg, img_file, overwrite=True)
    export_landmark_file(mimg.landmarks['PTS'], pts_file, overwrite=True)
    
    # Remove
    del img, mimg

100%|██████████| 192/192 [00:58<00:00,  3.30it/s]


In [13]:
ls /data1/famface01/data/stimuli/pics/fampics_exp_masked/*

/data1/famface01/data/stimuli/pics/fampics_exp_masked/Angelina_Jolie:
Angelina_Jolie_117.jpg  Angelina_Jolie_181.jpg  Angelina_Jolie_3_2.jpg
Angelina_Jolie_117.pts  Angelina_Jolie_181.pts  Angelina_Jolie_3_2.pts
Angelina_Jolie_128.jpg  Angelina_Jolie_187.jpg  Angelina_Jolie_330.jpg
Angelina_Jolie_128.pts  Angelina_Jolie_187.pts  Angelina_Jolie_330.pts
Angelina_Jolie_14.jpg   Angelina_Jolie_188.jpg  Angelina_Jolie_424.jpg
Angelina_Jolie_14.pts   Angelina_Jolie_188.pts  Angelina_Jolie_424.pts
Angelina_Jolie_153.jpg  Angelina_Jolie_189.jpg  Angelina_Jolie_4_2.jpg
Angelina_Jolie_153.pts  Angelina_Jolie_189.pts  Angelina_Jolie_4_2.pts
Angelina_Jolie_15.jpg   Angelina_Jolie_190.jpg  Angelina_Jolie_49.jpg
Angelina_Jolie_15.pts   Angelina_Jolie_190.pts  Angelina_Jolie_49.pts
Angelina_Jolie_168.jpg  Angelina_Jolie_21.jpg   Angelina_Jolie_70.jpg
Angelina_Jolie_168.pts  Angelina_Jolie_21.pts   Angelina_Jolie_70.pts
Angelina_Jolie_178.jpg  Angelina_Jolie_227.jpg
Angelina_Jolie_178.pts  Angelina_Jo

In [5]:
pwd

u'/data1/famface01/command/misc/face_representations/120_features/300_fampics'